In [1]:
import jax
from jax.example_libraries import stax, optimizers
import jax.numpy as jnp
from jax import grad, value_and_grad, vmap, random, jit, lax

from jax.config import config
import jax.numpy as jnp

from jax.nn.initializers import glorot_normal, normal, zeros, ones

import pandas as pd
import numpy as np

from scipy import sparse

import time

config.update("jax_enable_x64", True)


In [ ]:
#AlgorithmType = 0

#case = 8

# key_noise_train, key_noise_test =  random.split(random.PRNGKey(2))
#key_noise_train = random.PRNGKey(case + 10)
#key_noise_test = random.PRNGKey(case + 20)

#Net_work_key = 100

## 1. Loading data by pandas

In [ ]:
# Neural network models
#num_train = 500
#num_test = 500


#train_input_file_name =  'poisson_2D_state_obs_train_o10_d' + str(num_train)+ '_n12_AC_1_1_pt5'
#train_output_file_name = 'poisson_2D_parameter_train_d' + str(num_train)+ '_n12_AC_1_1_pt5'
#test_input_file_name =   'poisson_2D_state_obs_test_o10_d' + str(num_test)+ '_n12_AC_1_1_pt5'
#test_output_file_name =  'poisson_2D_parameter_test_d' + str(num_test)+ '_n12_AC_1_1_pt5'


#df_train_Observations = pd.read_csv('data/' + train_input_file_name + '.csv') 
#df_train_Parameters = pd.read_csv('data/' + train_output_file_name + '.csv')
#df_test_Observations = pd.read_csv('data/' + test_input_file_name + '.csv') 
#df_test_Parameters = pd.read_csv('data/' + test_output_file_name + '.csv')


#train_Observations_synthetic = np.reshape(df_train_Observations.to_numpy(), (num_train,-1))
#train_Parameters = np.reshape(df_train_Parameters.to_numpy(), (num_train,-1))
#test_Observations_synthetic = np.reshape(df_test_Observations.to_numpy(), (num_test,-1))
#true_Parameters = np.reshape(df_test_Parameters.to_numpy(), (num_test,-1))



In [ ]:
#df_Parameters = np.loadtxt('inverse_solution/Noise_0_Inverse_BFGS.txt', delimiter = ',')
#INV = np.reshape(df_Parameters, (num_test,-1))

#df_Parameters = np.loadtxt('inverse_solution/Noise_0_nDec.txt', delimiter = ',')
#nDec = np.reshape(df_Parameters, (num_test,-1))

#df_Parameters = np.loadtxt('inverse_solution/Noise_0_mcDec.txt', delimiter = ',')
#mcDec = np.reshape(df_Parameters, (num_test,-1))

#df_Parameters = np.loadtxt('inverse_solution/Noise_0_nFFNN.txt', delimiter = ',')
#nFFNN = np.reshape(df_Parameters, (num_test,-1))

#df_Parameters = np.loadtxt('inverse_solution/Noise_0_mcFFNN.txt', delimiter = ',')
#mcFFNN = np.reshape(df_Parameters, (num_test,-1))


### 1.0 Load Eigenvalue, Eigenvectors, observed indices, prematrices

In [3]:
# Physical model information
n = 9
dimension_of_PoI = (n + 1)**2 # external force field
num_observation = 10 # number of observed points
num_truncated_series = 12

df_Eigen = pd.read_csv('data/prior_mean_n12_AC_1_1_pt5' + '.csv') 
df_Sigma = pd.read_csv('data/prior_covariance_n12_AC_1_1_pt5' + '.csv') 

Eigen = np.reshape(df_Eigen.to_numpy(), (dimension_of_PoI, num_truncated_series))
Sigma = np.reshape(df_Sigma.to_numpy(), (num_truncated_series, num_truncated_series))

df_obs = pd.read_csv('data/poisson_2D_obs_indices_o10_n12' + '.csv')
obs_indices = np.reshape(df_obs.to_numpy(), (num_observation,-1))


boundary_matrix = sparse.load_npz('data/boundary_matrix_n12' + '.npz')
pre_mat_stiff_sparse = sparse.load_npz('data/prestiffness_n12' + '.npz')
load_vector_n12 = sparse.load_npz('data/load_vector_n12' + '.npz')
load_vector = sparse.csr_matrix.todense(load_vector_n12).T


df_free_index = pd.read_csv('data/prior_covariance_cholesky_n12_AC_1_1_pt5' + '.csv')
free_index = df_free_index.to_numpy()


i = 0
obs_transfered = []
for free_ind in free_index:
    if (free_ind in obs_indices):
        obs_transfered.append(i)
    i += 1

jjj = 0
obs_operator = np.zeros((num_observation, free_index.shape[0]))
for obs_index in obs_transfered:
    
    obs_operator[jjj, obs_index] = 1
    jjj+= 1
    
obs_operator = jax.numpy.asarray(obs_operator)  
Eigen = jax.numpy.asarray(Eigen)
Sigma = jax.numpy.asarray(Sigma)
pre_mat_stiff = jax.numpy.asarray(pre_mat_stiff_sparse.todense())

# 4. Testing / Making predictions on test samples

In [4]:
# Let's import all needed stuff
import jax
from jax.config import config

import jax.numpy as np
import numpy as onp

from scipy.optimize import minimize
# Library for automated PDE solution
import fenics
# Library for automated derivative computation of FEniCS programs
import fenics_adjoint
# UFL is domain specific language (DSL) for declaration of finite element discretizations of variational forms
import ufl 

# Suppress JIT compile message from FEniCS
import logging
logging.getLogger('FFC').setLevel(logging.WARNING)
logging.getLogger('UFL').setLevel(logging.WARNING)

# This is the core function here
from jaxfenics_adjoint import build_jax_fem_eval
from jaxfenics_adjoint import from_jax

import matplotlib.pyplot as plt
plt.ioff() # Turn interactive plotting off
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import matplotlib.tri as tri
import matplotlib.tri as mtri
import matplotlib.cm as cm

import numpy as np

import matplotlib

# font = {'family' : 'Times New Roman',
#     'weight' : 'normal',
#     'size'   : 12}

# matplotlib.rc('font', **font)
# plt.rcParams["font.family"] = "Times New Roman"

config.update("jax_enable_x64", True)
fenics.set_log_level(fenics.LogLevel.ERROR)

In [5]:
# Create mesh
mesh = fenics_adjoint.UnitSquareMesh(n, n)
V = fenics.FunctionSpace(mesh, "P", 1)

In [6]:
def convert_array_to_dolfin_function(V, nodal_values):
    nodal_values_dl = fenics.Function(V)
    nodal_values_dl.vector().set_local(np.squeeze(nodal_values))

    return nodal_values_dl

def plot_fem_function_fenics_2d(function_space, nodal_values,
                                title, filepath,
                                fig_size, colorbar_limits, type_color):

    #=== Convert array to dolfin function ===#
    nodal_values_fe = convert_array_to_dolfin_function(function_space, nodal_values)

    #=== Extract mesh and triangulate ===#
    mesh = nodal_values_fe.function_space().mesh()
    coords = mesh.coordinates()
    elements = mesh.cells()
    triangulation = tri.Triangulation(coords[:, 0], coords[:, 1], elements)

    #=== Plot figure ===#
    nodal_values = nodal_values_fe.compute_vertex_values(mesh)
    plt.figure()
    ax = plt.gca()

    #-----------------------------------------------------------------------------
    # Refine data
    #-----------------------------------------------------------------------------
    refiner = tri.UniformTriRefiner(triangulation)
    tri_refi, z_test_refi = refiner.refine_field(nodal_values, subdiv=3)

    #-----------------------------------------------------------------------------
    # Plot the triangulation and the high-res iso-contours
    #-----------------------------------------------------------------------------
    ax.set_aspect('equal')
    ax.triplot(triangulation, lw=0.5, color='w')
    levels = np.arange(colorbar_limits[0], colorbar_limits[1], colorbar_limits[2])
    cmap = cm.get_cmap(name=type_color, lut=None)
    tcf = ax.tricontourf(tri_refi, z_test_refi, levels=levels, cmap=cmap)
    plt.colorbar(tcf)

    for point in range(0, len(obs_indices)):
        plt.plot(obs_coords[point,0], obs_coords[point,1], 'gs', mew=3, ms=4)
    ax.set_xticks([])
    ax.set_yticks([])
    plt.savefig(filepath, dpi = 1200, bbox_inches = 'tight')
    # plt.close()

def forward(parameter_of_interest):
    
    # stiff = jnp.dot(pre_mat_stiff, jnp.exp(parameter_of_interest))
    stiff = sp_matmul(values, rows, cols, jnp.exp(parameter_of_interest), dimension_of_PoI**2)
    stiff = jnp.reshape(stiff, (dimension_of_PoI, dimension_of_PoI))
    
    A = jnp.squeeze(jnp.take(stiff,free_index, axis=1))

    B = jnp.squeeze(jnp.take(A,free_index, axis=0))
    
    state = jnp.linalg.solve(B, load_vector)
    Solution = np.zeros((dimension_of_PoI,1))
    for i in range(free_index.shape[0]):
        Solution[free_index[i,0]] = state[i]
        
    return Solution

## Plot the mesh grid and observed point

In [7]:
#=== Convert array to dolfin function ===#
node_value_vec  = np.zeros((100,1))
nodal_values_fe = convert_array_to_dolfin_function(V, node_value_vec)

#=== Extract mesh and triangulate ===#
mesh = nodal_values_fe.function_space().mesh()
coords = mesh.coordinates()
nodes = V.tabulate_dof_coordinates()
elements = mesh.cells()
triangulation = tri.Triangulation(coords[:, 0], coords[:, 1], elements)

df_obs = pd.read_csv('data/poisson_2D_obs_indices_o10_n12' + '.csv')
obs_indices = np.reshape(df_obs.to_numpy(), (num_observation,-1))

obs_coords = np.zeros((len(obs_indices), 2))
for point in range(0, len(obs_indices)):
    obs_coords[point,:] = nodes[obs_indices[point],:]
#=== Plot figure ===#
plt.figure()
ax = plt.gca()


ax.set_aspect('equal')
ax.triplot(triangulation, lw=0.5, color='b')
for point in range(0, len(obs_indices)):
    plt.plot(obs_coords[point,0], obs_coords[point,1], 'ro', mew=3, ms=4)
    

plt.savefig('figs/Observation_mesh2000.png', dpi = 2000, bbox_inches = 'tight')
plt.close()


## Ploting a traing sample (not for now)

In [9]:
# PLOTTING
import pandas as pd
import numpy as np

labels_test=pd.read_csv("labels_test.csv")


num_test=500

max = 2.00
min = 0.00
de = 0.1


df_test_Parameters = labels_test


true_Parameters = np.reshape(df_test_Parameters.to_numpy(), (num_test,-1))

df_Parameters = labels_test

INV = np.reshape(df_Parameters.to_numpy(), (num_test,-1))


def error(true, preds):
    delta_ab = jnp.abs(Eigen @ Sigma @ jnp.reshape(true - preds, (12,1)))**2 / jnp.mean(jnp.square(Eigen @ Sigma @ jnp.reshape(true, (12,1))))
    return delta_ab

n = 9
dimension_of_PoI = (n + 1)**2 # external force field
num_observation = 10 # number of observed points
num_truncated_series = 12

df_Eigen = pd.read_csv('data/prior_mean_n12_AC_1_1_pt5' + '.csv') 
df_Sigma = pd.read_csv('data/prior_covariance_n12_AC_1_1_pt5' + '.csv') 

Eigen = np.reshape(df_Eigen.to_numpy(), (dimension_of_PoI, num_truncated_series))
Sigma = np.reshape(df_Sigma.to_numpy(), (num_truncated_series, num_truncated_series))

Error = vmap(error, in_axes=(0, 0))


Average_Err = np.mean(Error(true_Parameters, 0*INV), axis = 0)

plot_fem_function_fenics_2d(V, np.abs(Average_Err.squeeze()), ' ', 'figs/Noise_0_Average_INV.png',(5,5), (min, max, de), 'YlOrBr')